In [1]:
from astropy.convolution import convolve
import numpy as np
import pandas as pd

In [2]:
Y =[[1,5,2,9,3,2],
    [6,0,7,3,9,3],
    [7,2,3,9,8,2],
    [2,1,4,0,5,2],
    [5,2,8,9,2,4]]

Y = np.array(Y)
np.squeeze(Y)

array([[1, 5, 2, 9, 3, 2],
       [6, 0, 7, 3, 9, 3],
       [7, 2, 3, 9, 8, 2],
       [2, 1, 4, 0, 5, 2],
       [5, 2, 8, 9, 2, 4]])

In [3]:
Y = np.where(Y==0, np.nan, Y)
Y

array([[ 1.,  5.,  2.,  9.,  3.,  2.],
       [ 6., nan,  7.,  3.,  9.,  3.],
       [ 7.,  2.,  3.,  9.,  8.,  2.],
       [ 2.,  1.,  4., nan,  5.,  2.],
       [ 5.,  2.,  8.,  9.,  2.,  4.]])

In [4]:
kernel = [[1,1,1],
          [1,1,1],
          [1,1,1]]

# 0 -- that cell is not included to calculate the mini grid mean

In [5]:
#Mean of Neigbouring elements: Won't work effectively for edges. 
##So choose satellite image that has a little bigger boundary than the focus area.
means = convolve(Y,kernel)
np.squeeze(means)

array([[1.5       , 2.625     , 3.25      , 3.66666667, 3.22222222,
        1.88888889],
       [2.625     , 4.125     , 5.        , 5.88888889, 5.33333333,
        3.        ],
       [2.25      , 4.        , 4.14285714, 6.        , 5.125     ,
        3.22222222],
       [2.11111111, 3.77777778, 4.75      , 6.        , 5.125     ,
        2.55555556],
       [1.11111111, 2.44444444, 3.        , 3.5       , 2.75      ,
        1.44444444]])

In [6]:
29/7

4.142857142857143

In [7]:
#Let's write a function for this:
def grid_mean(Y):
    Y = np.array(Y)
    #Replace 0s with np.nan
    Y = np.where(Y==0, np.nan, Y)
    
    kernel = [[1,1,1],
              [1,1,1],
              [1,1,1]]

    means = convolve(Y,kernel)

    means = np.squeeze(means)
    
    means[np.isnan(means)] = 0
    return means


In [8]:
df = pd.read_csv('data/indiasubcont_satpm_allyears.csv')[['ix', 'iy','Y1999']].reset_index()
df

,index,ix,iy,Y1999
0,0,1,1,0.0
1,1,2,1,0.0
2,2,3,1,0.0
3,3,4,1,0.0
4,4,5,1,0.0
...,...,...,...,...
102395,102395,316,320,14.5
102396,102396,317,320,14.5
102397,102397,318,320,15.1
102398,102398,319,320,16.7


In [9]:
df_pivot = df.pivot_table(columns='iy',index='ix',values='Y1999')
df_pivot

iy,1,2,3,4,5,6,7,8,9,10,...,311,312,313,314,315,316,317,318,319,320
ix,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20.5,20.5,28.2,29.0,31.2,31.8,30.9,31.5,26.0,20.4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21.3,21.2,29.1,30.5,30.6,31.3,30.5,30.8,28.7,23.5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,23.6,20.5,27.0,30.6,30.6,30.7,30.5,30.2,29.9,25.9
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,24.1,20.7,25.5,28.7,30.0,30.0,29.8,29.6,29.5,29.3
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.5,21.0,22.6,27.6,29.2,28.7,29.1,28.9,28.8,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.4,...,6.1,6.5,7.6,7.5,8.2,9.5,9.9,11.3,11.9,14.5
317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.4,...,6.3,6.7,8.2,7.6,8.4,9.4,10.1,11.2,13.2,14.5
318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.5,0.0,0.0,...,6.2,6.8,8.4,7.8,8.8,9.6,10.2,12.8,13.0,15.1


In [10]:
means = grid_mean(df_pivot)

In [11]:
df_9by9means = pd.DataFrame(means)
df_9by9means.columns = df_pivot.columns
df_9by9means.index = df_pivot.index


In [15]:
df_9by9means

iy,1,2,3,4,5,6,7,8,9,10,...,311,312,313,314,315,316,317,318,319,320
ix,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.000000,...,14.522222,15.644444,17.611111,19.844444,20.488889,20.700000,20.755556,19.822222,17.877778,10.955556
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.000000,...,22.211111,23.544444,26.288889,29.644444,30.700000,30.900000,30.911111,29.888889,27.433333,17.155556
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.000000,...,22.966667,23.666667,25.977778,29.177778,30.333333,30.444444,30.377778,29.944444,28.600000,18.533333
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.000000,...,24.144444,23.388889,24.911111,27.977778,29.566667,29.844444,29.722222,29.588889,29.011111,19.155556
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.000000,...,25.688889,23.533333,24.311111,26.788889,28.622222,28.877778,28.733333,28.888889,28.855556,19.233333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.50,16.925,17.171429,...,6.166667,6.755556,7.222222,7.800000,8.377778,9.211111,10.233333,11.222222,12.622222,8.844444
317,0.0,0.0,0.0,0.0,0.0,0.0,16.5,16.50,17.100,17.460000,...,6.244444,6.977778,7.455556,8.055556,8.533333,9.344444,10.444444,11.511111,13.055556,9.133333
318,0.0,0.0,0.0,0.0,0.0,0.0,16.5,16.50,16.950,17.375000,...,6.288889,7.077778,7.611111,8.244444,8.644444,9.611111,10.766667,11.977778,13.544444,9.511111


In [18]:
df_9by9means = pd.melt(df_9by9means.reset_index(), id_vars=['ix'], value_name='Y1999')
df_9by9means

,ix,iy,Y1999
0,1,1,0.000000
1,2,1,0.000000
2,3,1,0.000000
3,4,1,0.000000
4,5,1,0.000000
...,...,...,...
102395,316,320,8.844444
102396,317,320,9.133333
102397,318,320,9.511111
102398,319,320,10.000000


In [19]:
df_9by9means.to_csv('Y1999_9x9avg.csv')